In [ ]:
import os
import pandas as pd
import numpy as np
import plotly.express as px

def wage_table(df, id_col="ID", wage_col="WAGE", naics_col="NAICS2"):
    if naics_col not in df.columns:
        raise ValueError("naics_col not found")
    if df[naics_col].dtype.kind in "iufc":
        df = df.copy()
        df["NAICS2"] = df[naics_col].astype("Int64")
    else:
        df = df.copy()
        df["NAICS2"] = df[naics_col].astype(str).str.extract(r"(\d{2})", expand=False)
    bins = [0, 2500, 6250, 12500, 18750, 25000, 50000, 125000, 250000, np.inf]
    labels = ["1-2499","2500-6249","6250-12499","12500-18749","18750-24999","25000-49999","50000-124999","125000-249999","250000+"]
    df[wage_col] = pd.to_numeric(df[wage_col], errors="coerce")
    df["wage_class"] = pd.cut(df[wage_col], bins=bins, labels=labels, right=True, include_lowest=False)
    g = df.groupby(["NAICS2","wage_class"]).agg(total_wages=(wage_col,"sum"), id_count=(id_col,"nunique"))
    out = (g.unstack("wage_class").reindex(columns=pd.MultiIndex.from_product([["total_wages","id_count"], labels])).fillna(0).reset_index())
    out.columns = ["NAICS2"] + [f"{wc}__{metric}" for metric, wc in out.columns if wc]
    paired = []
    for wc in labels:
        for metric in ["id_count", "total_wages"]:
            c = f"{wc}__{metric}"
            if c in out.columns:
                paired.append(c)
    totals = df.groupby("NAICS2").agg(all_total_wages=(wage_col,"sum"), all_id_count=(id_col,"nunique")).reset_index()
    out = totals.merge(out, on="NAICS2", how="left")
    out = out[["NAICS2","all_total_wages","all_id_count"] + paired]
    return out

def build_master_wage_table(root_dir, id_col="ID", wage_col="WAGE", naics_col="NAICS2"):
    frames = []
    for dirpath, _, filenames in os.walk(root_dir):
        for f in filenames:
            if not f.lower().endswith(".csv"):
                continue
            fp = os.path.join(dirpath, f)
            try:
                df = pd.read_csv(fp)
            except Exception:
                continue
            base = os.path.splitext(f)[0]
            parts = base.split("_")
            if len(parts) >= 2 and parts[-2].isdigit() and parts[-1].isdigit():
                yr_qtr = f"{parts[-2]}Q{parts[-1]}"
            else:
                yr_qtr = base
            tbl = wage_table(df, id_col=id_col, wage_col=wage_col, naics_col=naics_col)
            tbl.insert(0, "yr_qtr", yr_qtr)
            frames.append(tbl)
    if not frames:
        cols = ["yr_qtr","NAICS2","all_total_wages","all_id_count","1-2499__id_count","1-2499__total_wages","2500-6249__id_count","2500-6249__total_wages","6250-12499__id_count","6250-12499__total_wages","12500-18749__id_count","12500-18749__total_wages","18750-24999__id_count","18750-24999__total_wages","25000-49999__id_count","25000-49999__total_wages","50000-124999__id_count","50000-124999__total_wages","125000-249999__id_count","125000-249999__total_wages","250000+__id_count","250000+__total_wages"]
        return pd.DataFrame(columns=cols)
    return pd.concat(frames, ignore_index=True)

def plot_wage_viz(master: pd.DataFrame, view="bubble", value="all_id_count", naics2=None, save_html=None):
    df = master.copy()
    if view == "bubble":
        df["avg_wage"] = df["all_total_wages"] / df["all_id_count"].replace(0, pd.NA)
        fig = px.scatter(df, x="avg_wage", y="all_id_count", size="all_total_wages", color="NAICS2", animation_frame="yr_qtr", hover_data=["NAICS2","yr_qtr","all_total_wages","all_id_count"], size_max=60, labels={"avg_wage":"Average wage","all_id_count":"ID count","all_total_wages":"Total wages"}, title="Avg Wage vs ID Count (size = Total Wages)")
    elif view == "stacked":
        if value not in {"all_id_count","all_total_wages"}:
            raise ValueError
        fig = px.bar(df, x="yr_qtr", y=value, color="NAICS2", barmode="stack", labels={"yr_qtr":"Quarter", value:value.replace("_"," ").title()}, title=f"Stacked: {value.replace('_',' ').title()} by NAICS2 per Quarter")
    elif view == "heatmap":
        if value not in {"all_id_count","all_total_wages"}:
            raise ValueError
        p = df.pivot(index="NAICS2", columns="yr_qtr", values=value).fillna(0)
        fig = px.imshow(p, labels=dict(x="Quarter", y="NAICS2", color=value.replace("_"," ").title()), aspect="auto", title=f"Heatmap: {value.replace('_',' ').title()} (NAICS2 × Quarter)")
    elif view == "band_shares":
        if naics2 is None:
            raise ValueError
        d = df[df["NAICS2"] == naics2].copy()
        bin_cols = [c for c in d.columns if c.endswith("__id_count")]
        d["id_total"] = d["all_id_count"].replace(0, pd.NA)
        for c in bin_cols:
            d[c.replace("__id_count","__id_share")] = d[c] / d["id_total"]
        share_cols = [c for c in d.columns if c.endswith("__id_share")]
        long = d.melt(id_vars=["yr_qtr"], value_vars=share_cols, var_name="band", value_name="share")
        long["band"] = long["band"].str.replace("__id_share","", regex=False)
        fig = px.line(long.sort_values("yr_qtr"), x="yr_qtr", y="share", color="band", labels={"share":"Share of employees","yr_qtr":"Quarter","band":"Wage band"}, title=f"Wage Band Shares Over Time — NAICS2 {naics2}")
    else:
        raise ValueError
    if save_html:
        fig.write_html(save_html, include_plotlyjs="cdn", full_html=True)
    fig.show()
    return fig

root_dir = "path/to/csv_folder"
master = build_master_wage_table(root_dir, id_col="ID", wage_col="WAGE", naics_col="NAICS2")
master.head()

plot_wage_viz(master, view="bubble")
plot_wage_viz(master, view="stacked", value="all_id_count")
plot_wage_viz(master, view="stacked", value="all_total_wages")
plot_wage_viz(master, view="heatmap", value="all_id_count")
plot_wage_viz(master, view="heatmap", value="all_total_wages")
plot_wage_viz(master, view="band_shares", naics2=str(master["NAICS2"].dropna().astype(str).iloc[0]))
